In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Налаштування стилю графіків
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🏠 Аналіз цін на нерухомість за допомогою Lasso Regression")
print("=" * 60)

# Завантаження даних про нерухомість в Каліфорнії
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Створюємо DataFrame для зручності
feature_names = housing.feature_names
df = pd.DataFrame(X, columns=feature_names)
df['Price'] = y

print(f"📊 Розмір датасету: {df.shape}")
print(f"📋 Ознаки: {list(feature_names)}")
print(f"📈 Середня ціна: ${np.mean(y):.2f} (в сотнях тисяч)")

# Створюємо додаткові ознаки (polynomial features) для демонстрації селекції
np.random.seed(42)
n_samples, n_features = X.shape

# Додаємо взаємодії між ознаками та квадратичні терми
interactions = []
interaction_names = []

for i in range(n_features):
    for j in range(i+1, n_features):
        interactions.append((X[:, i] * X[:, j]).reshape(-1, 1))
        interaction_names.append(f"{feature_names[i]} × {feature_names[j]}")

# Квадратичні терми
for i in range(n_features):
    interactions.append((X[:, i] ** 2).reshape(-1, 1))
    interaction_names.append(f"{feature_names[i]}²")

# Додаємо шумні ознаки (нерелевантні)
noise_features = []
noise_names = []
for i in range(10):
    noise_features.append(np.random.normal(0, 1, (n_samples, 1)))
    noise_names.append(f"Noise_{i+1}")

# Об'єднуємо всі ознаки
X_extended = np.hstack([X] + interactions + noise_features)
all_feature_names = list(feature_names) + interaction_names + noise_names

print(f"🔧 Розширений набір ознак: {X_extended.shape[1]} ознак")

# Розділяємо дані
X_train, X_test, y_train, y_test = train_test_split(
    X_extended, y, test_size=0.2, random_state=42
)

# Стандартизація
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Cross-validation для вибору оптимального alpha
print("\n🔍 Пошук оптимального параметра регуляризації...")
alphas = np.logspace(-4, 1, 50)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=2000)
lasso_cv.fit(X_train_scaled, y_train)

optimal_alpha = lasso_cv.alpha_
print(f"🎯 Оптимальний alpha: {optimal_alpha:.6f}")

# 2. Тренування моделей
# Lasso з оптимальним alpha
lasso = Lasso(alpha=optimal_alpha, max_iter=2000, random_state=42)
lasso.fit(X_train_scaled, y_train)

# Звичайна лінійна регресія для порівняння
linear_reg = LinearRegression()
linear_reg.fit(X_train_scaled, y_train)

# 3. Прогнози та оцінки
y_pred_lasso = lasso.predict(X_test_scaled)
y_pred_linear = linear_reg.predict(X_test_scaled)

lasso_mse = mean_squared_error(y_test, y_pred_lasso)
linear_mse = mean_squared_error(y_test, y_pred_linear)

lasso_r2 = r2_score(y_test, y_pred_lasso)
linear_r2 = r2_score(y_test, y_pred_linear)

print(f"\n📈 Результати моделей:")
print(f"Lasso Regression - MSE: {lasso_mse:.4f}, R²: {lasso_r2:.4f}")
print(f"Linear Regression - MSE: {linear_mse:.4f}, R²: {linear_r2:.4f}")

# 4. Аналіз коефіцієнтів та селекції ознак
coefficients = pd.DataFrame({
    'Feature': all_feature_names,
    'Lasso_Coef': lasso.coef_,
    'Linear_Coef': linear_reg.coef_
})

# Фільтруємо ненульові коефіцієнти Lasso
non_zero_features = coefficients[coefficients['Lasso_Coef'] != 0].copy()
non_zero_features['Abs_Lasso_Coef'] = np.abs(non_zero_features['Lasso_Coef'])
non_zero_features = non_zero_features.sort_values('Abs_Lasso_Coef', ascending=False)

print(f"\n🎯 Lasso вибрав {len(non_zero_features)} з {len(all_feature_names)} ознак:")
print(non_zero_features[['Feature', 'Lasso_Coef']].head(10))

# Візуалізація
fig = plt.figure(figsize=(20, 15))

# 1. Cross-validation кривя для alpha
plt.subplot(2, 3, 1)
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_.mean(axis=1), 'b-', alpha=0.6)
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_.mean(axis=1) + lasso_cv.mse_path_.std(axis=1), 'b--', alpha=0.3)
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_.mean(axis=1) - lasso_cv.mse_path_.std(axis=1), 'b--', alpha=0.3)
plt.axvline(optimal_alpha, color='red', linestyle='--', label=f'Optimal α = {optimal_alpha:.6f}')
plt.xlabel('Alpha (Regularization Parameter)')
plt.ylabel('Mean Squared Error')
plt.title('Cross-Validation для вибору Alpha')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Порівняння прогнозів
plt.subplot(2, 3, 2)
plt.scatter(y_test, y_pred_lasso, alpha=0.6, label=f'Lasso (R² = {lasso_r2:.3f})', s=30)
plt.scatter(y_test, y_pred_linear, alpha=0.6, label=f'Linear (R² = {linear_r2:.3f})', s=30)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Справжні ціни')
plt.ylabel('Прогнозовані ціни')
plt.title('Порівняння прогнозів моделей')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Коефіцієнти найважливіших ознак
plt.subplot(2, 3, 3)
top_features = non_zero_features.head(15)
colors = ['green' if coef > 0 else 'red' for coef in top_features['Lasso_Coef']]
bars = plt.barh(range(len(top_features)), top_features['Lasso_Coef'], color=colors, alpha=0.7)
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Коефіцієнт Lasso')
plt.title('Топ-15 найважливіших ознак')
plt.grid(True, alpha=0.3)
plt.gca().invert_yaxis()

# 4. Розподіл коефіцієнтів
plt.subplot(2, 3, 4)
plt.hist(coefficients['Lasso_Coef'], bins=50, alpha=0.7, label='Lasso', edgecolor='black')
plt.axvline(0, color='red', linestyle='--', alpha=0.8)
plt.xlabel('Значення коефіцієнтів')
plt.ylabel('Частота')
plt.title('Розподіл коефіцієнтів Lasso')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Кількість ненульових коефіцієнтів vs Alpha
plt.subplot(2, 3, 5)
alphas_range = np.logspace(-4, 1, 30)
n_nonzero = []
for alpha in alphas_range:
    lasso_temp = Lasso(alpha=alpha, max_iter=2000)
    lasso_temp.fit(X_train_scaled, y_train)
    n_nonzero.append(np.sum(lasso_temp.coef_ != 0))

plt.semilogx(alphas_range, n_nonzero, 'bo-', markersize=4)
plt.axvline(optimal_alpha, color='red', linestyle='--', label=f'Optimal α')
plt.xlabel('Alpha')
plt.ylabel('Кількість ненульових коефіцієнтів')
plt.title('Селекція ознак залежно від Alpha')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Залишки моделі
plt.subplot(2, 3, 6)
residuals = y_test - y_pred_lasso
plt.scatter(y_pred_lasso, residuals, alpha=0.6, s=30)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Прогнозовані значення')
plt.ylabel('Залишки')
plt.title('Аналіз залишків Lasso моделі')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Додатковий аналіз: порівняння з різними значеннями alpha
print(f"\n🔬 Додатковий аналіз впливу регуляризації:")
alphas_demo = [0.001, optimal_alpha, 0.1, 1.0]
results_table = []

for alpha in alphas_demo:
    lasso_temp = Lasso(alpha=alpha, max_iter=2000)
    lasso_temp.fit(X_train_scaled, y_train)
    y_pred_temp = lasso_temp.predict(X_test_scaled)
    mse_temp = mean_squared_error(y_test, y_pred_temp)
    r2_temp = r2_score(y_test, y_pred_temp)
    n_features_temp = np.sum(lasso_temp.coef_ != 0)
    
    results_table.append({
        'Alpha': alpha,
        'MSE': mse_temp,
        'R²': r2_temp,
        'N_Features': n_features_temp
    })

results_df = pd.DataFrame(results_table)
print(results_df.round(4))

print(f"\n✅ Висновки:")
print(f"• Lasso автоматично вибрав {len(non_zero_features)} найважливіших ознак з {len(all_feature_names)}")
print(f"• Видалив усі 10 шумних ознак та багато нерелевантних взаємодій")
print(f"• Покращив інтерпретабельність моделі без значної втрати точності")
print(f"• Оптимальний alpha ({optimal_alpha:.6f}) забезпечує баланс між точністю та простотою")

🏠 Аналіз цін на нерухомість за допомогою Lasso Regression
📊 Розмір датасету: (20640, 9)
📋 Ознаки: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
📈 Середня ціна: $2.07 (в сотнях тисяч)
🔧 Розширений набір ознак: 54 ознак

🔍 Пошук оптимального параметра регуляризації...
🎯 Оптимальний alpha: 0.006866

📈 Результати моделей:
Lasso Regression - MSE: 0.5584, R²: 0.5739
Linear Regression - MSE: 0.4650, R²: 0.6452

🎯 Lasso вибрав 20 з 54 ознак:
                  Feature  Lasso_Coef
6                Latitude   -0.783592
7               Longitude   -0.743563
0                  MedInc    0.442935
14     MedInc × Longitude   -0.274965
8       MedInc × HouseAge    0.187547
15    HouseAge × AveRooms   -0.136438
36                MedInc²   -0.135636
29  AveBedrms × Longitude   -0.120073
16   HouseAge × AveBedrms    0.116827
10     MedInc × AveBedrms    0.094880
